In [140]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os
import matplotlib.pyplot as plt

In [141]:
import os
import shutil
import random

def split_data(source_dir, train_dir, test_dir, test_size=0.2):
    # پوشه‌های گربه و سگ
    class_names = ['cat', 'dog']
    
    # ایجاد پوشه‌های train و test
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)
    
    # تقسیم داده‌ها برای هر کلاس
    for class_name in class_names:
        class_dir = os.path.join(source_dir, class_name)
        
        # بررسی وجود پوشه
        if not os.path.exists(class_dir):
            print(f"پوشه {class_name} در مسیر {source_dir} وجود ندارد.")
            continue
        
        files = os.listdir(class_dir)
        random.shuffle(files)
        
        # تعداد داده‌ها برای تست
        test_count = int(len(files) * test_size)
        
        # داده‌های تست و آموزش
        test_files = files[:test_count]
        train_files = files[test_count:]
        
        # ایجاد پوشه‌های مربوط به کلاس‌ها در پوشه‌های train و test
        os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
        os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)
        
        # کپی داده‌ها به پوشه‌های مربوطه
        for file_name in train_files:
            shutil.copy(os.path.join(class_dir, file_name), os.path.join(train_dir, class_name, file_name))
        for file_name in test_files:
            shutil.copy(os.path.join(class_dir, file_name), os.path.join(test_dir, class_name, file_name))

# مسیر اصلی داده‌ها
source_dir = 'PetImages'  # مسیر داده‌ها باید درست باشد

# مسیرهای داده‌های آموزش و تست
train_dir = 'dataset/train'
test_dir = 'dataset/test'

# تقسیم داده‌ها به آموزش و تست
split_data(source_dir, train_dir, test_dir, test_size=0.2)


In [139]:
#  تعیین device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)
#  پیش‌پردازش تصاویر
transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

#  بارگذاری داده‌ها
train_data = datasets.ImageFolder('dataset/train', transform=transform)
test_data = datasets.ImageFolder('dataset/test', transform=transform)

train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
test_loader = DataLoader(test_data, batch_size=16, shuffle=False)

Using device: cpu


In [143]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        
        # Fully Connected (Dense) layers
        self.fc1 = nn.Linear(256 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 2)  # 2 کلاس: سگ و گربه

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # -> [32, 75, 75]
        x = self.pool(F.relu(self.conv2(x)))  # -> [64, 37, 37]
        x = self.pool(F.relu(self.conv3(x)))  # -> [128, 18, 18]
        x = self.pool(F.relu(self.conv4(x)))  # -> [256, 9, 9]
        x = self.pool(F.relu(self.conv5(x)))  # -> [256, 5, 5]
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)  # بدون softmax (استفاده از CrossEntropy)
        return x
model = CNNModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [144]:
# حلقه آموزش
for epoch in range(5):
    model.train()
    running_loss = 0.0
    total = 0
    correct = 0
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{epoch}, Loss: {running_loss:.4f}, Accuracy: {acc:.2f}%")


Epoch 1/0, Loss: 1020.9075, Accuracy: 60.11%
Epoch 2/1, Loss: 763.5458, Accuracy: 76.60%
Epoch 3/2, Loss: 583.2753, Accuracy: 83.25%
Epoch 4/3, Loss: 452.0194, Accuracy: 87.71%
Epoch 5/4, Loss: 369.8821, Accuracy: 90.10%


In [145]:
# 7. تست مدل
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 93.16%
